# Module 1 + Module 2 (no tuple)
The problem is formally defined as follows:
- a depot 0 where all vehicles are located in the beginning
- a set of customers $C = {1,...,N}$, we define locations $L = {0} + C$
- a demand $d_i$ for each customer $i \in C$ (the depot has no demand)
- a time window $[a_i,b_i]$ with $0<a_i<b_i$ for each location $i \in L$ (including the depot)
- a service time $s_i$ for each customer $i \in C$ (the depot has zero service time)
- costs $c_{ij}$ and time $t_{ij}$ for travelling from location $i$ to location $j$
- a homogeneous fleet $K = {1,...,|K|}$ of vehicles with load capacity $Q$

We want to find
- a route for each vehicle, i.e., an ordered sequence of customers
- with minimal total routing costs/time, such that
- each customer is visited exactly once
- the start times of the services have to be within the depot and customer time windows (potentially introducing waiting times between services),
- and the vehicle capacity is never exceeded

## Instance Configuration

In [1]:
import math
import random

### INSTANCE CONFIGURATION

numCustomers = 5
maxNumVehicles = 2

vehicleCapacity = 30
demandRange = (2, 5)

timeHorizon = 100 # tmax
timeWindowWidthRange = (10, 15)
serviceTimeRange = (2, 4)

## Instance Generation

In [2]:
random.seed(0)

depot = 0
customers = [*range(1, numCustomers + 1)]
locations = [depot] + customers
connections = [(i, j) for i in locations for j in locations if i != j]
vehicles = [*range(1, maxNumVehicles + 1)]

# create random depot and customer locations in the Euclidian plane (1000x1000)
points = [(random.randint(0, 999), random.randint(0, 999)) for i in locations]
# dictionary of Euclidean distance for each connection (interpreted as travel costs)
costs = {
    (i, j): math.ceil(
        math.sqrt(sum((points[i][k] - points[j][k]) ** 2 for k in range(2)))
    )
    for (i, j) in connections
}
maximalCosts = math.ceil(999 * math.sqrt(2))
# dictionary of travel times for each connection (related to the costs, scaled to time horizon)
travelTimes = {
    (i, j): math.ceil((costs[i, j] / maximalCosts) * timeHorizon * 0.2)
    for (i, j) in connections
}

# create random demands, service times, and time window widths in the given range
demands = {i: random.randint(demandRange[0], demandRange[1]) for i in customers}
demands[0] = 0  # depot has no demand
serviceTimes = {
    i: random.randint(serviceTimeRange[0], serviceTimeRange[1]) for i in customers
}
serviceTimes[0] = 0  # depot has no service time
timeWindowWidths = {
    i: random.randint(timeWindowWidthRange[0], timeWindowWidthRange[1])
    for i in customers
}
# vehicles are allowed to leave the depot any time within the time horizon
timeWindowWidths[0] = timeHorizon

# create time windows randomly based on the previously generated information
# such that the service at a customer can be finished within the time horizon
timeWindows = {}
timeWindows[0] = (0, 0 + timeWindowWidths[0])
for i in customers:
    start = random.randint(0, timeHorizon - serviceTimes[i] - timeWindowWidths[i] - travelTimes[i,0])
    timeWindows[i] = (start, start + timeWindowWidths[i])

## Output Visualisation

In [3]:
print("Customer Demands:")
for customer, demand in demands.items():
    print(f"Customer {customer}: Demand = {demand}")

print("\nCustomer Service Times:")
for customer, service_time in serviceTimes.items():
    print(f"Customer {customer}: Service Time = {service_time}")

print("\nCustomer Time Windows:")
for customer, window in timeWindows.items():
    print(f"Customer {customer}: Time Window = {window}")

print("\nTravel Times:")
for (i, j), time in list(travelTimes.items()):
    print(f"Travel Time from {i} to {j}: {time}")


Customer Demands:
Customer 1: Demand = 4
Customer 2: Demand = 5
Customer 3: Demand = 4
Customer 4: Demand = 3
Customer 5: Demand = 3
Customer 0: Demand = 0

Customer Service Times:
Customer 1: Service Time = 3
Customer 2: Service Time = 2
Customer 3: Service Time = 2
Customer 4: Service Time = 4
Customer 5: Service Time = 3
Customer 0: Service Time = 0

Customer Time Windows:
Customer 0: Time Window = (0, 100)
Customer 1: Time Window = (12, 26)
Customer 2: Time Window = (9, 24)
Customer 3: Time Window = (42, 56)
Customer 4: Time Window = (60, 71)
Customer 5: Time Window = (71, 83)

Travel Times:
Travel Time from 0 to 1: 8
Travel Time from 0 to 2: 8
Travel Time from 0 to 3: 12
Travel Time from 0 to 4: 6
Travel Time from 0 to 5: 11
Travel Time from 1 to 0: 8
Travel Time from 1 to 2: 14
Travel Time from 1 to 3: 8
Travel Time from 1 to 4: 7
Travel Time from 1 to 5: 6
Travel Time from 2 to 0: 8
Travel Time from 2 to 1: 14
Travel Time from 2 to 3: 14
Travel Time from 2 to 4: 7
Travel Time fr

## Model Generation
### Basic Route Problem (without Load and Time constraint)
Decision variables:
- $x_{ij} = 
\begin{cases} 
1 & \text{if a vehicle traverses arc (i, j)} \\
0 & \text{otherwise}
\end{cases}
$

The objective is to minimize the total travel time. Travel time doesn't consider the service time of each customer.

Constraints:
1. Each customer is visited exactly once, i.e., there is exactly one incoming connection and exactly one    nextCustomer connection.
2. At most $|K|$ vehicles can be used.
3. We can also compute a lower bound on the number of needed vehicles, that is the sum of all demands divided by the vehicle capacity.

$$
\text{min} \sum_{i,j \in \mathcal{L}} c_{ij}x_{ij}
$$

$$
\sum_{j \in \mathcal{C}} x_{ij} = 1 \quad \forall j \in \mathcal{C}
$$

$$
\sum_{j \in \mathcal{L}} x_{ij} = 1 \quad \forall i \in \mathcal{C}
$$

$$
\sum_{j \in \mathcal{C}} x_{0j} \leq |K|
$$

$$
\sum_{j \in \mathcal{C}} x_{0j} \geq \left\lceil \frac{\sum_{i \in \mathcal{C}} d_i}{Q} \right\rceil
$$


In [4]:
import gurobipy as gp
from gurobipy import GRB

# Function for calculate the number of vehicles needed for customers
def numVehiclesNeededForCustomers(customers):
    sumDemand = 0
    for i in customers:
        sumDemand += demands[i]
    return math.ceil(sumDemand / vehicleCapacity)

model = gp.Model("VRPTW")

# binary variables x(i,j): is 1 if some vehicle is going from node i to node j, 0 otherwise
x = model.addVars(connections, vtype=GRB.BINARY, name="x")

# objective function: minimize sum of connection costs
model.setObjective(gp.quicksum(travelTimes[i,j]*x[i,j] for i,j in connections), GRB.MINIMIZE)

# all customers have exactly one incoming and one outgoing connection
model.addConstrs((x.sum("*", j) == 1 for j in customers), name="incoming")
model.addConstrs((x.sum(i, "*") == 1 for i in customers), name="outgoing")

# vehicle limits
model.addConstr(x.sum(0, "*") <= maxNumVehicles, name="maxNumVehicles")
model.addConstr(
    x.sum(0, "*") >= numVehiclesNeededForCustomers(customers),
    name="minNumVehicles",
)


Set parameter Username
Academic license - for non-commercial use only - expires 2025-04-02


<gurobi.Constr *Awaiting Model Update*>

Then we can add Load and Time constraints in two different ways:

## Big - M Model

### Load

Decision Variable: $y_i$ for each location $i \in L$ to denote the vehicle load after picking up the demand at $i$.

Constraints:
- $y_0 = 0$
- $y_i + d_j \le y_j + Q(1 - x_{ij}) \quad \forall i \in L, j \in C, i \neq j$
- $y_i + d_j \geq y_j - M_{ij}(1 - x_{ij}) \quad \forall i \in L, j \in C, i \neq j$

### Time

Decision Variable: $z_i$ for each location $i \in L$ to denote the start time of the service at $i$ that has to be within its time window.

Constraints:
- $z_0 = 0$
- $z_i + s_i + t_{ij} \leq z_j + T(1 - x_{ij}) \quad \forall i \in L, j \in C, i \neq j$

In both case, subtours are eliminated since time/load has to monotonically increase along a route.

## Flow Model
Now, we extend the meaning of the variables used in the Big-M model. We do not associate resource states to customers anymore but instead to connections between customers.

### Load 

Decision variable: $y_{i,j}$ for each connection $(i, j)$ to denote the vehicle load after picking up the demand at $i$ and proceeding to location $j$.

Constraints:
 $$y_{0,j} = 0 \quad \forall j \in C$$
 $$\sum_{i \in L} y_{ij} + d_j = \sum_{i \in L} y_{ji} \quad \forall j \in C$$
 $$y_{ij} \geq d_ix_{ij} \quad \forall i \in C, j \in L, i \neq j$$
 $$y_{ij} \leq (Q - d_j)x_{ij} \quad \forall i \in C, j \in L, i \neq j$$

### Time
Decision variable: $z_{ij}$ for each connection $(i, j)$ to denote the start time of the service at $i$ (which must be within its time window) when immediately proceeding to location $j$.

Constraints:
 $$z_{0j} = 0 \quad \forall j \in C$$
 $$\sum_{i \in L}[z_{ij} + (s_i + t_{ij})x_{ij}] \leq \sum_{i \in L} z_{ji} \quad \forall j \in C$$
 $$z_{ij}\geq a_ix_{ij} \quad \forall i,j \in L, i \neq j$$
 $$z_{ij}\leq b_ix_{ij} \quad \forall i,j \in L, i \neq j$$

In [5]:
def addLoadConstraintsByBigM():

    y = model.addVars(locations, lb=0, ub=vehicleCapacity, name="y")
    y[0].UB = 0  # empty load at depot

    model.addConstrs(
        (
            y[i] + demands[j] <= y[j] + vehicleCapacity * (1 - x[i, j])
            for i in locations
            for j in customers
            if i != j
        ),
        name="loadBigM1",
    )
    model.addConstrs(
        (
            y[i] + demands[j]
            >= y[j] - (vehicleCapacity - demands[i] - demands[j]) * (1 - x[i, j])
            for i in locations
            for j in customers
            if i != j
        ),
        name="loadBigM2",
    )


def addLoadConstraintsByFlows():

    y = model.addVars(connections, lb=0, ub=vehicleCapacity, name="y")

    for i in customers:
        y[0, i].UB = 0

    model.addConstrs(
        (y.sum("*", j) + demands[j] == y.sum(j, "*") for j in customers),
        name="flowConservation",
    )
    model.addConstrs(
        (
            y[i, j] >= demands[i] * x[i, j]
            for i in customers
            for j in locations
            if i != j
        ),
        name="loadLowerBound",
    )
    model.addConstrs(
        (
            y[i, j] <= (vehicleCapacity - demands[j]) * x[i, j]
            for i in customers
            for j in locations
            if i != j
        ),
        name="loadUpperBound",
    )


def addTimeConstraintsByBigM():

    z = model.addVars(locations, name="z")
    for i in locations:
        z[i].LB = timeWindows[i][0]
        z[i].UB = timeWindows[i][1]

    model.addConstrs(
        (
            z[i] + serviceTimes[i] + travelTimes[i, j]
            <= z[j]
            + (
                timeWindows[i][1]
                + serviceTimes[i]
                + travelTimes[i, j]
                - timeWindows[j][0]
            )
            * (1 - x[i, j])
            for i in locations
            for j in customers
            if i != j
        ),
        name="timeBigM",
    )


def addTimeConstraintsByFlows():

    z = model.addVars(connections, lb=0, name="z")

    for (i, j) in connections:
        z[i, j].UB = timeWindows[i][1]

    model.addConstrs(
        (
            gp.quicksum(
                z[i, j] + (serviceTimes[i] + travelTimes[i, j]) * x[i, j]
                for i in locations
                if (i, j) in connections
            )
            <= z.sum(j, "*")
            for j in customers
        ),
        name="flowConservation",
    )
    model.addConstrs(
        (
            z[i, j] >= timeWindows[i][0] * x[i, j]
            for i in customers
            for j in locations
            if i != j
        ),
        name="timeWindowStart",
    )
    model.addConstrs(
        (
            z[i, j] <= timeWindows[i][1] * x[i, j]
            for i in customers
            for j in locations
            if i != j
        ),
        name="timeWindowEnd",
    )


In [6]:
### MODEL CONFIGURATION
loadModelType = 1  # 1: big-M, 2: flow
timeModelType = 1  # 1: big-M, 2: flow

if loadModelType == 1:
    addLoadConstraintsByBigM()
elif loadModelType == 2:
    addLoadConstraintsByFlows()

if timeModelType == 1:
    addTimeConstraintsByBigM()
elif timeModelType == 2:
    addTimeConstraintsByFlows()

#model.params.Threads = 4
model.optimize()

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[x86] - Darwin 23.6.0 23G93)

CPU model: Intel(R) Core(TM) i3-1000NG4 CPU @ 1.10GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 87 rows, 42 columns and 285 nonzeros
Model fingerprint: 0x6298a233
Variable types: 12 continuous, 30 integer (30 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [3e+00, 1e+01]
  Bounds range     [1e+00, 1e+02]
  RHS range        [1e+00, 9e+01]
Found heuristic solution: objective 57.0000000
Presolve removed 64 rows and 28 columns
Presolve time: 0.00s
Presolved: 23 rows, 14 columns, 87 nonzeros
Variable types: 3 continuous, 11 integer (11 binary)

Root relaxation: objective 5.100000e+01, 6 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0      51.

## Output Generation

In [7]:
if model.SolCount >= 1:

    print("Objective Value: ", model.objVal)

    usedConnections = [(i, j) for (i, j) in x.keys() if x[i, j].X > 0.5]

    # create a dict for the next customer based on the current one
    # (note that the depot in general has multiple outgoing connections)
    nextCustomer = {}
    for (i, j) in usedConnections:
        if i == 0:
            if 0 not in nextCustomer.keys():
                nextCustomer[0] = []
            nextCustomer[0].append(j)
        else:
            nextCustomer[i] = j

    print(f"Solution contains {len(nextCustomer[0])} routes:")
    routeNumber = 0
    visitedCustomers = [False] * (numCustomers + 1)
    for firstCustomer in nextCustomer[0]:
        print(f"Route {routeNumber}: 0 -> ", end="")
        vehicleLoad = 0
        time = travelTimes[0, firstCustomer]
        violatedTimeWindows = False
        currentCustomer = firstCustomer
        while currentCustomer != 0:
            print(f"{currentCustomer} (L:{vehicleLoad}, T:{time}) -> ", end="")
            visitedCustomers[currentCustomer] = True
            vehicleLoad += demands[currentCustomer]
            time = max(time, timeWindows[currentCustomer][0])
            if time > timeWindows[currentCustomer][1]:
                violatedTimeWindows = True
            time += (
                serviceTimes[currentCustomer]
                + travelTimes[currentCustomer, nextCustomer[currentCustomer]]
            )
            currentCustomer = nextCustomer[currentCustomer]
        print(f"0 (L:{vehicleLoad}/{vehicleCapacity}, T:{time})")
        if vehicleLoad > vehicleCapacity:
            print("Vehicle capacity is exceeded!")
        if violatedTimeWindows:
            print("Time windows are violated!")
        routeNumber += 1

    print("Unvisited customers: ", end="")
    for c in customers:
        if visitedCustomers[c] == False:
            print(f"{c}, ", end="")

Objective Value:  51.0
Solution contains 2 routes:
Route 0: 0 -> 1 (L:0, T:8) -> 3 (L:4, T:23) -> 5 (L:8, T:47) -> 0 (L:11/30, T:85)
Route 1: 0 -> 2 (L:0, T:8) -> 4 (L:5, T:18) -> 0 (L:8/30, T:70)
Unvisited customers: 

T[i] tempo di arrivo al nodo i

z[i] tempo in cui puo iniziare il servizio in i

# Module 1 + Module 2 

## Input Generation

In [17]:
import gurobipy as gp
from gurobipy import GRB
import math
import random

### INSTANCE CONFIGURATION
numCustomers = 5
maxNumVehicles = 2

vehicleCapacity = 30
demandRange = (2, 5)

timeHorizon = 100
timeWindowWidthRange = (10, 15)
serviceTimeRange = (2, 4)
random.seed(0)

depot = 0
customers = [*range(1, numCustomers + 1)]
locations = [depot] + customers
connections = [(i, j) for i in locations for j in locations if i != j]
vehicles = [*range(1, maxNumVehicles + 1)]

# create random depot and customer locations in the Euclidian plane (1000x1000)
points = [(random.randint(0, 999), random.randint(0, 999)) for i in locations]
# dictionary of Euclidean distance for each connection (interpreted as travel costs)
costs = {
    (i, j): math.ceil(
        math.sqrt(sum((points[i][k] - points[j][k]) ** 2 for k in range(2)))
    )
    for (i, j) in connections
}
maximalCosts = math.ceil(999 * math.sqrt(2))
# dictionary of travel times for each connection (related to the costs, scaled to time horizon)
travelTimes = {
    (i, j): math.ceil((costs[i, j] / maximalCosts) * timeHorizon * 0.2)
    for (i, j) in connections
}

# create random demands, service times, and time window widths in the given range
demands = {i: random.randint(demandRange[0], demandRange[1]) for i in customers}
demands[0] = 0  # depot has no demand
serviceTimes = {
    i: random.randint(serviceTimeRange[0], serviceTimeRange[1]) for i in customers
}
serviceTimes[0] = 0  # depot has no service time
timeWindowWidths = {
    i: random.randint(timeWindowWidthRange[0], timeWindowWidthRange[1])
    for i in customers
}
# vehicles are allowed to leave the depot any time within the time horizon
timeWindowWidths[0] = timeHorizon

# create time windows randomly based on the previously generated information
# such that the service at a customer can be finished within the time horizon
timeWindows = {}
timeWindows[0] = (0, 0 + timeWindowWidths[0])
for i in customers:
    start = random.randint(0, timeHorizon - serviceTimes[i] - timeWindowWidths[i] - travelTimes[i,0])
    timeWindows[i] = (start, start + timeWindowWidths[i])

def numVehiclesNeededForCustomers(customers):
    sumDemand = 0
    for i in customers:
        sumDemand += demands[i]
    return math.ceil(sumDemand / vehicleCapacity)

# Define the restricted tuples R
R = [
    (1,3,5)
]

## Model Generation

In [18]:
model = gp.Model("VRPTW")

# Decision Variables
x = model.addVars(connections, vtype=GRB.BINARY, name="x")
y = model.addVars(locations, lb=0, ub=vehicleCapacity, name="y")
y[0].UB = 0
z = model.addVars(locations, name="z")

# Objective Function
model.setObjective(gp.quicksum(travelTimes[i,j]*x[i,j] for i,j in connections), GRB.MINIMIZE)

# all customers have exactly one incoming and one outgoing connection
model.addConstrs((x.sum("*", j) == 1 for j in customers), name="incoming")
model.addConstrs((x.sum(i, "*") == 1 for i in customers), name="outgoing")

# vehicle limits
model.addConstr(x.sum(0, "*") <= maxNumVehicles, name="maxNumVehicles")
model.addConstr(
    x.sum(0, "*") >= numVehiclesNeededForCustomers(customers),
    name="minNumVehicles",
)

model.addConstrs(
    (
        y[i] + demands[j] <= y[j] + vehicleCapacity * (1 - x[i, j])
        for i in locations
        for j in customers
        if i != j
    ),
    name="loadBigM1",
)
model.addConstrs(
    (
        y[i] + demands[j]
        >= y[j] - (vehicleCapacity - demands[i] - demands[j]) * (1 - x[i, j])
        for i in locations
        for j in customers
        if i != j
    ),
    name="loadBigM2",
)

for i in locations:
    z[i].LB = timeWindows[i][0]
    z[i].UB = timeWindows[i][1]

model.addConstrs(
    (
        z[i] + serviceTimes[i] + travelTimes[i, j]
        <= z[j]
        + (
            timeWindows[i][1]
            + serviceTimes[i]
            + travelTimes[i, j]
            - timeWindows[j][0]
        )
        * (1 - x[i, j])
        for i in locations
        for j in customers
        if i != j
    ),
    name="timeBigM",
)

# Add the new constraint for the set R
for (i, j, l) in R:
    model.addConstr(
        gp.quicksum(x[i, k] for k in customers if k != i and k != j) + 
        gp.quicksum(x[k, j] for k in customers if k != i and k != j) - 
        1 >= x[i, j] + x[j, l] - 1,
        name=f"no_shared_vehicle_{i}_{j}_{l}"
    )

model.params.Threads = 4
model.optimize()

Set parameter Threads to value 4
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[x86] - Darwin 23.6.0 23G93)

CPU model: Intel(R) Core(TM) i3-1000NG4 CPU @ 1.10GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 88 rows, 42 columns and 293 nonzeros
Model fingerprint: 0xe68445c6
Variable types: 12 continuous, 30 integer (30 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [3e+00, 1e+01]
  Bounds range     [1e+00, 1e+02]
  RHS range        [1e+00, 9e+01]
Found heuristic solution: objective 61.0000000
Presolve removed 80 rows and 38 columns
Presolve time: 0.00s
Presolved: 8 rows, 4 columns, 24 nonzeros
Variable types: 0 continuous, 4 integer (4 binary)
Found heuristic solution: objective 57.0000000

Explored 0 nodes (0 simplex iterations) in 0.02 seconds (0.00 work units)
Thread count was 4 (of 4 available processors)

Solution count 2: 57 61 

Optimal solution found (tolerance 1.00e-04)
Best ob

## Output Generation

In [19]:
if model.SolCount >= 1:

    print("Objective Value: ", model.objVal)

    usedConnections = [(i, j) for (i, j) in x.keys() if x[i, j].X > 0.5]

    # create a dict for the next customer based on the current one
    # (note that the depot in general has multiple outgoing connections)
    nextCustomer = {}
    for (i, j) in usedConnections:
        if i == 0:
            if 0 not in nextCustomer.keys():
                nextCustomer[0] = []
            nextCustomer[0].append(j)
        else:
            nextCustomer[i] = j

    print(f"Solution contains {len(nextCustomer[0])} routes:")
    routeNumber = 0
    visitedCustomers = [False] * (numCustomers + 1)
    for firstCustomer in nextCustomer[0]:
        print(f"Route {routeNumber}: 0 -> ", end="")
        vehicleLoad = 0
        time = travelTimes[0, firstCustomer]
        violatedTimeWindows = False
        currentCustomer = firstCustomer
        while currentCustomer != 0:
            print(f"{currentCustomer} (L:{vehicleLoad}, T:{time}) -> ", end="")
            visitedCustomers[currentCustomer] = True
            vehicleLoad += demands[currentCustomer]
            time = max(time, timeWindows[currentCustomer][0])
            if time > timeWindows[currentCustomer][1]:
                violatedTimeWindows = True
            time += (
                serviceTimes[currentCustomer]
                + travelTimes[currentCustomer, nextCustomer[currentCustomer]]
            )
            currentCustomer = nextCustomer[currentCustomer]
        print(f"0 (L:{vehicleLoad}/{vehicleCapacity}, T:{time})")
        if vehicleLoad > vehicleCapacity:
            print("Vehicle capacity is exceeded!")
        if violatedTimeWindows:
            print("Time windows are violated!")
        routeNumber += 1

    print("Unvisited customers: ", end="")
    for c in customers:
        if visitedCustomers[c] == False:
            print(f"{c}, ", end="")

Objective Value:  57.0
Solution contains 2 routes:
Route 0: 0 -> 1 (L:0, T:8) -> 4 (L:4, T:22) -> 0 (L:7/30, T:70)
Route 1: 0 -> 2 (L:0, T:8) -> 3 (L:5, T:25) -> 5 (L:9, T:47) -> 0 (L:12/30, T:85)
Unvisited customers: 

# Module 3

In [121]:
from gurobipy import Model, GRB, quicksum

def solve_vrppl(customers, lockers, depot, K, C, c_ij, d_i, Q, a_i, b_i, t_ij, s_i, 
                N_HC, N_LC, N_HLC, N_PL, N_C, M):
    # Create a new model
    model = Model("VRPPL")

    # Sets and parameters
    N = list(range(len(C)))  # Set of nodes (customers, depot, lockers)
    # Variables
    x = model.addVars(N, N, K, vtype=GRB.BINARY, name="x")  # Binary decision: vehicle k travels from i to j
    h = model.addVars(N_C, vtype=GRB.BINARY, name="h")      # Binary decision: customer i is delivered home
    l = model.addVars(N_C, vtype=GRB.BINARY, name="l")      # Binary decision: customer i is delivered to locker
    y = model.addVars(N_C, N_PL, vtype=GRB.BINARY, name="y") # Binary decision: customer i is assigned to locker j
    psi = model.addVars(N_PL, K, vtype=GRB.CONTINUOUS, name="psi") # Packages delivered to locker station j by vehicle k
    mu = model.addVars(N_C, K, vtype=GRB.CONTINUOUS, name="mu") # Time vehicle k starts serving node i
    partial_k = model.addVars(K, vtype=GRB.CONTINUOUS, name="partial") # Time vehicle k leaves depot
    nu = model.addVars(K, vtype=GRB.CONTINUOUS, name="nu") # Time vehicle k returns to depot

    # Objective function (1): Minimize the total traveling cost
    model.setObjective(quicksum(c_ij[i, j] * x[i, j, k] for k in range(K) for i in N for j in N), GRB.MINIMIZE)

    # Constraints
    # (2) Each HC customer must be served exactly once by one vehicle
    model.addConstrs(quicksum(x[i, j, k] for k in range(K) for j in N) == 1 for i in N_HC)

    # (3) Each HLC customer may be served at home at most once
    model.addConstrs(quicksum(x[i, j, k] for k in range(K) for j in N) <= h[i] for i in N_HLC)

    # (4) LC customers cannot be delivered at home
    model.addConstrs(quicksum(x[i, j, k] for k in range(K) for j in N) == 0 for i in N_LC)

    # (5) Each vehicle can leave the depot at most once
    model.addConstrs(quicksum(x[depot, j, k] for j in N_C) <= 1 for k in range(K))

    # (6) Vehicle flow conservation at each node
    model.addConstrs(quicksum(x[i, j, k] for i in N if i != j) - quicksum(x[j, i, k] for i in N if i != j) == 0 for k in range(K) for j in N)

    # (7) Each vehicle returns to the depot at most once
    model.addConstrs(quicksum(x[i, depot, k] for i in N_C) <= 1 for k in range(K))

    # (8) Vehicle capacity constraint
    model.addConstrs(quicksum(d_i[j] * x[i, j, k] for i in N for j in N_C if i != j) + 
                     quicksum(psi[m, k] for p in N for m in N_PL if p != m) <= Q for k in range(K))

    # (9) Departure time from depot
    model.addConstrs(partial_k[k] + t_ij[depot, i] - mu[i, k] <= M * (1 - x[depot, i, k]) for k in range(K) for i in N_C)

    # (10) Time windows and service time constraint
    model.addConstrs(mu[i, k] + s_i[i] + t_ij[i, j] - mu[j, k] <= M * (1 - x[i, j, k]) for k in range(K) for i in N_C for j in N_C if i != j)

    # (11) Time to return to depot
    model.addConstrs(mu[i, k] - nu[k] <= M * (1 - x[i, depot, k]) for k in range(K) for i in N_C)

    # (12) Home delivery within time window
    model.addConstrs(quicksum(mu[i, k] * h[i] for k in range(K)) >= a_i[i] * h[i] for i in N_C)

    model.addConstrs(quicksum(mu[i, k] * h[i] for k in range(K)) <= b_i[i] * h[i] for i in N_C)
    
    # (13) Either home delivery or locker, not both
    model.addConstrs(h[i] + l[i] == 1 for i in N_C)

    # (14) HC customers must receive home delivery
    model.addConstrs(h[i] == 1 for i in N_HC)

    # (15) LC customers must be assigned to a locker
    model.addConstrs(l[i] == 1 for i in N_LC)

    # (16) Locker assignment for customers
    model.addConstrs(quicksum(y[i, j] for j in N_PL) == l[i] for i in N_C)

    # (17) Selection of parcel locker
    model.addConstrs(y[i, j] <= 1 for i in N_C for j in N_PL)

    # (18) Capacity of parcel lockers
    model.addConstrs(quicksum(y[i, j] * d_i[i] for i in N_LC + N_HLC) == quicksum(psi[j, k] for k in range(K)) for j in N_PL)

    # (19) Locker capacity
    model.addConstrs(quicksum(y[i, j] for i in N_C) <= 1 for j in N_PL)

    # Optimize the model
    model.optimize()

    # Extract the results
    if model.status == GRB.OPTIMAL:
        print('Optimal objective:', model.objVal)
        solution = model.getAttr('x', x)
        return solution
    else:
        print("No optimal solution found")
        return None

In [120]:
# Numero di clienti e locker
customers = list(range(1, 6))  # 5 clienti
lockers = list(range(6, 8))  # 2 locker
depot = 0  # Nodo del deposito

# Numero di veicoli
K = 2

# Set di nodi (clienti, deposito, locker)
C = customers + lockers + [depot]

# Costo di viaggio tra i nodi (esempio casuale)
c_ij = {(i, j): abs(i - j) * 10 for i in C for j in C}

# Domanda di ciascun cliente (esempio casuale)
d_i = {i: (i + 1) * 2 for i in customers}

# Capacità dei veicoli
Q = 30

# Limiti di tempo per la consegna a casa dei clienti (tempo di arrivo minimo e massimo)
a_i = {i: 10 * i for i in customers}
b_i = {i: 15 * i for i in customers}

# Tempo di viaggio tra i nodi (esempio casuale)
t_ij = {(i, j): abs(i - j) * 5 for i in C for j in C}

# Tempo di servizio per ogni cliente (esempio casuale)
s_i = {i: 2 for i in customers}

# Sottogruppi di nodi
N_HC = []  # Clienti che devono essere serviti a casa
N_LC = []  # Clienti che devono ricevere tramite locker
N_HLC = [1, 2, 3, 4, 5]    # Clienti che possono ricevere tramite casa o locker
N_PL = lockers  # Locker
N_C = customers  # Clienti

# Un numero grande per i vincoli di tipo Big-M
M = 1000

# Chiamata alla funzione con questi parametri
solve_vrppl(customers, lockers, depot, K, C, c_ij, d_i, Q, a_i, b_i, t_ij, s_i, 
            N_HC, N_LC, N_HLC, N_PL, N_C, M)


Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[x86] - Darwin 23.6.0 23G93)

CPU model: Intel(R) Core(TM) i3-1000NG4 CPU @ 1.10GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 111 rows, 166 columns and 642 nonzeros
Model fingerprint: 0xb842aa19
Model has 10 quadratic constraints
Variable types: 18 continuous, 148 integer (148 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+01, 8e+01]
  Objective range  [1e+01, 7e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+03]
Presolve removed 20 rows and 31 columns
Presolve time: 0.01s
Presolved: 106 rows, 153 columns, 678 nonzeros
Presolved model has 12 SOS constraint(s)
Variable types: 26 continuous, 127 integer (127 binary)
Found heuristic solution: objective 0.0000000

Explored 0 nodes (0 simplex iterations) in 0.07 seconds (0.00 work units)
Thread count was 4 (of 4 available processors)



{(0, 0, 0): 0.0,
 (0, 0, 1): 0.0,
 (0, 1, 0): -0.0,
 (0, 1, 1): -0.0,
 (0, 2, 0): -0.0,
 (0, 2, 1): -0.0,
 (0, 3, 0): -0.0,
 (0, 3, 1): -0.0,
 (0, 4, 0): -0.0,
 (0, 4, 1): -0.0,
 (0, 5, 0): -0.0,
 (0, 5, 1): -0.0,
 (0, 6, 0): -0.0,
 (0, 6, 1): -0.0,
 (0, 7, 0): -0.0,
 (0, 7, 1): -0.0,
 (1, 0, 0): -0.0,
 (1, 0, 1): -0.0,
 (1, 1, 0): 0.0,
 (1, 1, 1): 0.0,
 (1, 2, 0): -0.0,
 (1, 2, 1): -0.0,
 (1, 3, 0): -0.0,
 (1, 3, 1): -0.0,
 (1, 4, 0): -0.0,
 (1, 4, 1): -0.0,
 (1, 5, 0): -0.0,
 (1, 5, 1): -0.0,
 (1, 6, 0): -0.0,
 (1, 6, 1): -0.0,
 (1, 7, 0): -0.0,
 (1, 7, 1): -0.0,
 (2, 0, 0): -0.0,
 (2, 0, 1): -0.0,
 (2, 1, 0): -0.0,
 (2, 1, 1): -0.0,
 (2, 2, 0): 0.0,
 (2, 2, 1): 0.0,
 (2, 3, 0): -0.0,
 (2, 3, 1): -0.0,
 (2, 4, 0): -0.0,
 (2, 4, 1): -0.0,
 (2, 5, 0): -0.0,
 (2, 5, 1): -0.0,
 (2, 6, 0): -0.0,
 (2, 6, 1): -0.0,
 (2, 7, 0): -0.0,
 (2, 7, 1): -0.0,
 (3, 0, 0): -0.0,
 (3, 0, 1): -0.0,
 (3, 1, 0): -0.0,
 (3, 1, 1): -0.0,
 (3, 2, 0): -0.0,
 (3, 2, 1): -0.0,
 (3, 3, 0): 0.0,
 (3, 3, 1): 0.0,


# Module 4

In [122]:
import gurobipy as gp
from gurobipy import GRB
import math
import random

### INSTANCE CONFIGURATION
numCustomers = 5
maxNumVehicles = 2

vehicleCapacity = 30
demandRange = (2, 5)

timeHorizon = 100
timeWindowWidthRange = (10, 15)
serviceTimeRange = (2, 4)
random.seed(0)

depot = 0
customers = [*range(1, numCustomers + 1)]
locations = [depot] + customers
connections = [(i, j) for i in locations for j in locations if i != j]
vehicles = [*range(1, maxNumVehicles + 1)]

# create random depot and customer locations in the Euclidian plane (1000x1000)
points = [(random.randint(0, 999), random.randint(0, 999)) for i in locations]
# dictionary of Euclidean distance for each connection (interpreted as travel costs)
costs = {
    (i, j): math.ceil(
        math.sqrt(sum((points[i][k] - points[j][k]) ** 2 for k in range(2)))
    )
    for (i, j) in connections
}
maximalCosts = math.ceil(999 * math.sqrt(2))
# dictionary of travel times for each connection (related to the costs, scaled to time horizon)
travelTimes = {
    (i, j): math.ceil((costs[i, j] / maximalCosts) * timeHorizon * 0.2)
    for (i, j) in connections
}

# create random demands, service times, and time window widths in the given range
demands = {i: random.randint(demandRange[0], demandRange[1]) for i in customers}
demands[0] = 0  # depot has no demand
serviceTimes = {
    i: random.randint(serviceTimeRange[0], serviceTimeRange[1]) for i in customers
}
serviceTimes[0] = 0  # depot has no service time
timeWindowWidths = {
    i: random.randint(timeWindowWidthRange[0], timeWindowWidthRange[1])
    for i in customers
}
# vehicles are allowed to leave the depot any time within the time horizon
timeWindowWidths[0] = timeHorizon

# create time windows randomly based on the previously generated information
# such that the service at a customer can be finished within the time horizon
timeWindows = {}
timeWindows[0] = (0, 0 + timeWindowWidths[0])
for i in customers:
    start = random.randint(0, timeHorizon - serviceTimes[i] - timeWindowWidths[i] - travelTimes[i,0])
    timeWindows[i] = (start, start + timeWindowWidths[i])

def numVehiclesNeededForCustomers(customers):
    sumDemand = 0
    for i in customers:
        sumDemand += demands[i]
    return math.ceil(sumDemand / vehicleCapacity)

# Define the restricted tuples R
R = [
    (1,3,5)
]

In [126]:
model = gp.Model("VRPTW")

# Decision Variables
x = model.addVars(connections, vtype=GRB.BINARY, name="x")
y = model.addVars(locations, lb=0, ub=vehicleCapacity, name="y")
y[0].UB = 0
z = model.addVars(locations, name="z")

# Variabili per il tempo di ritorno al deposito
return_times = model.addVars(range(maxNumVehicles), name="return_times")

# Objective Function 1: Minimize Travel Time
model.setObjective(gp.quicksum(travelTimes[i, j] * x[i, j] for i, j in connections), GRB.MINIMIZE)

# All customers have exactly one incoming and one outgoing connection
model.addConstrs((x.sum("*", j) == 1 for j in customers), name="incoming")
model.addConstrs((x.sum(i, "*") == 1 for i in customers), name="outgoing")

# Vehicle limits
model.addConstr(x.sum(0, "*") <= maxNumVehicles, name="maxNumVehicles")
model.addConstr(
    x.sum(0, "*") >= numVehiclesNeededForCustomers(customers),
    name="minNumVehicles",
)

model.addConstrs(
    (
        y[i] + demands[j] <= y[j] + vehicleCapacity * (1 - x[i, j])
        for i in locations
        for j in customers
        if i != j
    ),
    name="loadBigM1",
)
model.addConstrs(
    (
        y[i] + demands[j] >= y[j] - (vehicleCapacity - demands[i] - demands[j]) * (1 - x[i, j])
        for i in locations
        for j in customers
        if i != j
    ),
    name="loadBigM2",
)

for i in locations:
    z[i].LB = timeWindows[i][0]
    z[i].UB = timeWindows[i][1]

model.addConstrs(
    (
        z[i] + serviceTimes[i] + travelTimes[i, j] <= z[j] + 
        (timeWindows[i][1] + serviceTimes[i] + travelTimes[i, j] - timeWindows[j][0]) * (1 - x[i, j])
        for i in locations
        for j in customers
        if i != j
    ),
    name="timeBigM",
)

# Add the new constraint for the set R
for (i, j, l) in R:
    model.addConstr(
        gp.quicksum(x[i, k] for k in customers if k != i and k != j) + 
        gp.quicksum(x[k, j] for k in customers if k != i and k != j) - 
        1 >= x[i, j] + x[j, l] - 1,
        name=f"no_shared_vehicle_{i}_{j}_{l}"
    )

# Calculate return times for each vehicle
for v in range(maxNumVehicles):
    model.addConstr(
        return_times[v] == z[0] - timeWindows[0][0],  # Deviato in base al percorso specifico
        name=f"return_time_{v}"
    )

# Compute mean return time
mean_return_time = model.addVar(name="mean_return_time")
model.addConstr(
    mean_return_time == gp.quicksum(return_times[v] for v in range(maxNumVehicles)) / maxNumVehicles,
    name="mean_return_time_calculation"
)

# Variabili ausiliarie per la deviazione assoluta
deviations = model.addVars(range(maxNumVehicles), name="deviations")

# Calculate mean absolute deviation from mean return time
model.addConstrs(
    (deviations[v] >= return_times[v] - mean_return_time for v in range(maxNumVehicles)),
    name="mad_calculation_pos"
)
model.addConstrs(
    (deviations[v] >= mean_return_time - return_times[v] for v in range(maxNumVehicles)),
    name="mad_calculation_neg"
)

# Objective Function 2: Minimize Mean Absolute Deviation
mad = model.addVar(name="mad")
model.addConstr(mad == gp.quicksum(deviations[v] for v in range(maxNumVehicles)) / maxNumVehicles, name="mad_definition")
model.setObjectiveN(mad, index=1, priority=1, weight=1)

# Calculate optimal value of first objective after optimization
model.optimize()
optimal_travel_time = model.getObjective().getValue()

# Add constraint for 5% degradation
model.addConstr(
    optimal_travel_time * 0.95 >= gp.quicksum(travelTimes[i, j] * x[i, j] for i, j in connections),
    name="5_percent_degradation"
)

# Optimize the model again
model.params.Threads = 4
model.optimize()


Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[x86] - Darwin 23.6.0 23G93)

CPU model: Intel(R) Core(TM) i3-1000NG4 CPU @ 1.10GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 96 rows, 48 columns and 315 nonzeros
Model fingerprint: 0x35dee942
Variable types: 18 continuous, 30 integer (30 binary)
Coefficient statistics:
  Matrix range     [5e-01, 1e+02]
  Objective range  [1e+00, 1e+01]
  Bounds range     [1e+00, 1e+02]
  RHS range        [1e+00, 9e+01]

---------------------------------------------------------------------------
Multi-objectives: starting optimization with 2 objectives ... 
---------------------------------------------------------------------------

Multi-objectives: applying initial presolve ...
---------------------------------------------------------------------------

Presolve removed 78 rows and 31 columns
Presolve time: 0.01s
Presolved: 18 rows, 17 columns, 53 nonzeros
----------------------------------